In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import backend as K
from tensorflow.keras import layers, Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout,Conv2D, MaxPooling3D, Conv3D, MaxPooling2D, TimeDistributed,LSTM
from tensorflow.keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam,SGD,RMSprop
from tensorflow.keras.applications import VGG16, VGG19, ResNet50, Xception, InceptionV3,InceptionResNetV2

import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"


gpus = tf.config.experimental.list_physical_devices(device_type='GPU')
cpus = tf.config.experimental.list_physical_devices(device_type='CPU')
print(gpus, cpus)

for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)

import matplotlib.pyplot as plt

import numpy as np
import tensorflow_addons as tfa

In [ ]:

img_size = (150,150)
X = np.load("../Data/COVID CXR/covid_org6000_150_X.npy")
Y = np.load("../Data/COVID CXR/covid_org6000_150_Y.npy")

In [ ]:

index = [i for i in range(X.shape[0])]
np.random.shuffle(index) # 打乱索引


X2 = X[index]/255.0
Y2 = Y[index]

In [ ]:
Y3 = tf.keras.utils.to_categorical(Y2, 2)

In [ ]:
learning_rate = 0.001
weight_decay = 0.0001
batch_size = 32
num_epochs = 300
image_size = 150  # We'll resize input images to this size
patch_size = 16  # Size of the patches to be extract from the input images
num_patches = (image_size // patch_size) ** 2
projection_dim = 64
num_heads = 4
transformer_units = [
    projection_dim * 2,
    projection_dim,
]  # Size of the transformer layers
transformer_layers = 16
mlp_head_units = [2048, 1024]  # Size of the dense layers of the final classifier


In [ ]:
def mlp(x, hidden_units, dropout_rate):
    for units in hidden_units:
        x = layers.Dense(units, activation=tf.nn.gelu)(x)
        x = layers.Dropout(dropout_rate)(x)
    return x


In [ ]:
class Patches(layers.Layer):
    def __init__(self, patch_size):
        super(Patches, self).__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches


In [ ]:
class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super(PatchEncoder, self).__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(
            input_dim=num_patches, output_dim=projection_dim
        )

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded


In [ ]:
def create_vit_classifier():
    inputs = layers.Input(shape=input_shape)

    # Create patches.
    patches = Patches(patch_size)(inputs)
    # Encode patches.
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

    # Create multiple layers of the Transformer block.
    for _ in range(transformer_layers):
        # Layer normalization 1.
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        # Create a multi-head attention layer.
        attention_output = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=projection_dim, dropout=0.1
        )(x1, x1)
        # Skip connection 1.
        x2 = layers.Add()([attention_output, encoded_patches])
        # Layer normalization 2.
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        # MLP.
        x3 = mlp(x3, hidden_units=transformer_units, dropout_rate=0.1)
        # Skip connection 2.
        encoded_patches = layers.Add()([x3, x2])

    # Create a [batch_size, projection_dim] tensor.
    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = layers.Flatten()(representation)
    representation = layers.Dropout(0.5)(representation)
    # Add MLP.
    features = mlp(representation, hidden_units=mlp_head_units, dropout_rate=0.5)
    # Classify outputs.
    logits = layers.Dense(num_classes)(features)
    # Create the Keras model.
    model = keras.Model(inputs=inputs, outputs=logits)
    return model


In [ ]:
from sklearn.model_selection import train_test_split 
x_train, x_test, y_train, y_test = train_test_split(X2, Y2, test_size=0.2, random_state=1)

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from tensorflow.keras.callbacks import ReduceLROnPlateau,EarlyStopping
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.1,
                              patience=10)
earlystop = EarlyStopping(monitor='val_accuracy', patience=15)

In [ ]:
seed = 101
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
cvacc = []
cvauc1 = []

i = 1
for train, test in kfold.split(X2,Y2):
    


    
    num_classes = 2
    input_shape = (150, 150, 3)

    model = create_vit_classifier()

    optimizer = tfa.optimizers.AdamW(
        learning_rate=learning_rate, weight_decay=weight_decay
    )

    model.compile(
        optimizer=optimizer,
        loss=keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=[
            keras.metrics.CategoricalAccuracy(name="accuracy"),

        ],
    )



    filepath = "../Data/models/150_lung_{0}_vit.h5".format(str(i))
    if os.path.exists(filepath):
        os.remove(filepath)
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1,monitor='val_accuracy',mode='max'
    )

    model.fit(X2[train],Y3[train], validation_split=0.2,epochs=300, batch_size=32, verbose=1,shuffle=True,callbacks=[checkpoint,earlystop])
    model.load_weights(filepath)
    # evaluate the model
    evals = model.evaluate(X2[test],Y3[test], verbose=1)
    y_pred = model.predict(X2[test])

    auc = roc_auc_score(Y3[test],y_pred)
    with open('../Data/models/150_lung_{0}_vit.csv'.format(str(i)),'w') as file:
        file.write('true_label,predict_label'+'\n')
        for a,b in zip(Y2[test],y_pred):
            file.write(str(a) + ','+str(np.argmax(b))+'\n')
    acc = evals[1]
   
    
    print("%s: %.2f%%" % (model.metrics_names[1], acc))
    print("auc1: %.2f"%(auc))

    cvacc.append(acc*100)
    cvauc1.append(auc)

    i+=1
print("mean acc %.2f%% (+/- %.2f%%)" % (np.mean(cvacc), np.std(cvacc)))
print("mean auc1 %.2f%% (+/- %.2f%%)" % (np.mean(cvauc1), np.std(cvauc1)))


In [ ]:
cvacc

In [ ]:
np.mean(cvacc)

In [ ]:
cvauc1

In [ ]:
np.mean(cvauc1)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix,accuracy_score,f1_score,roc_auc_score,recall_score,precision_score

In [ ]:
def calculate_metric(gt, pred): 
    pred2 = []
    for i in pred:
        if i>0.5:
            pred2.append(1)
        else:
            pred2.append(0)
    confusion = confusion_matrix(gt,pred2)
    TP = confusion[1, 1]
    TN = confusion[0, 0]
    FP = confusion[0, 1]
    FN = confusion[1, 0]
    acc = (TP+TN)/float(TP+TN+FP+FN)
    auc = roc_auc_score(gt,pred)
    precision = TP/float(TP+FP)
    f1_score_ = f1_score(gt,pred2)
    sensitivity = TP / float(TP+FN)
    specificity = TN / float(TN+FP)
    
    return round(acc,4),round(auc,4),round(f1_score_,4), round(precision,4),round(sensitivity,4), round(specificity,4)

In [ ]:
df1 = pd.read_csv('../Data/models/150_lung_1_vit.csv')


calculate_metric(df1['true_label'],df1['predict_label'])

In [ ]:
df2 = pd.read_csv('../Data/models/150_lung_2_vit.csv')


calculate_metric(df2['true_label'],df2['predict_label'])

In [ ]:
df3 = pd.read_csv('../Data/models/150_lung_3_vit.csv')


calculate_metric(df3['true_label'],df3['predict_label'])

In [ ]:
df4 = pd.read_csv('../Data/models/150_lung_4_vit.csv')


calculate_metric(df4['true_label'],df4['predict_label'])

In [ ]:
df5 = pd.read_csv('../Data/models/150_lung_5_vit.csv')


calculate_metric(df5['true_label'],df5['predict_label'])